In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [5]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
X = df.drop(columns=['math_score'],axis=1)

In [7]:
y = df['math_score']

In [8]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [9]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
cat_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoding",cat_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [10]:
X=preprocessor.fit_transform(X)

In [11]:
X.shape

(1000, 19)

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [14]:
X_train

array([[ 0.        ,  1.        ,  0.        , ...,  1.        ,
         0.81073728,  0.19396819],
       [ 0.        ,  1.        ,  1.        , ...,  1.        ,
        -0.42274011, -0.39860266],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.08484337,  1.18158627],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.39957815,  0.2598094 ],
       [ 0.        ,  1.        ,  0.        , ...,  1.        ,
         0.39957815,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
        -0.35421359, -0.59612627]], shape=(800, 19))

Create an Evaluate Function to give all metrics after model Training

In [18]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [19]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3998
- Mean Absolute Error: 4.3200
- R2 Score: 0.8706
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.0648
- Mean Absolute Error: 3.9898
- R2 Score: 0.8955


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.6245
- Mean Absolute Error: 5.2418
- R2 Score: 0.8053
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.4650
- Mean Absolute Error: 5.0935
- R2 Score: 0.8297


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.4001
- Mean Absolute Error: 4.3195
- R2 Score: 0.8706
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.0657
- Mean Absolute Error: 3.9876
- R2 Score: 0.8954


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.9096
- Mean Absolute Error: 4.6985
- R2 Score: 0.8450
-----------------------

In [20]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.895481
2,Ridge,0.895447
7,CatBoosting Regressor,0.876273
5,Random Forest Regressor,0.856534
8,AdaBoost Regressor,0.844734
6,XGBRegressor,0.836210
1,Lasso,0.829706
3,K-Neighbors Regressor,0.805870
4,Decision Tree,0.749648
